# INSY 5336 PROJECT

## Algorithm

- 1: Install beautifulsoup 4 library.
- 2: Import all the necessary libraries: requests and re.
- 3: Open file 'stocks.csv' in write mode.
- 4: Using the requests lib get the url for scrapping: https://money.cnn.com/data/hotstocks/
- 5: Using beautifulsoup library , scrap the page using html.parser.
- 6: Extract ticker symbol and company name from parsed page and store it in dictionary.
- 7: Under each title, get the company information for each ticker symbol using loop.
- 8: Call function 'data_scrap' with ticker symbol as its argument.
- 9: 'data_scrap' function will scrape data from website https://in.finance.yahoo.com/quote/ as in step 4 and step 5.       It will return three values.
- 10: Write company name, ticker symbol, open price, previous close and volume in csv file. 
- 11: Close the file after writing all the values.
- 12: Ask user to enter one ticker symbol from the list.
- 13: If entered ticker symbol is found in the csv file, print the information of the company and finish the program.
- 14: Else, ask user to enter valid ticker symbol.

## Code Instructions :

- Execute cell which has Installation command for  beatifulsoup library at first. 
- After that, Start the program by selecting the cell. 
- Program will ask for user input. If the input is correct it will display the data and finish the program. To check for different results, run the program again.
- If user has entered wrong output, program will ask for input until it's correct.
- Input is NOT case sensitive.



## Expected Result :

        Enter your Interested Company :  ll

        You did not enter correct ticker symbol from the list.
    
        Enter your Interested Company :  AMD


        The data for  Advanced Micro Devices, Inc. (AMD) is the following:

        Advanced Micro Devices, Inc. 
        Prev close: 52.39
        Open:  51.07
        Volume:  68,930,002
     
        
        
        
## Note :

- There is a possibility that sometimes program terminates and results in output 'Run the program again'. This might happen because of of following reasons: 
- Internet connection :  Check if you are connected to the internet and then restart the kernel and try running the program again.
- Website data changes :  At a certain time in a day, data gets dynamically changed on the website because of which requested url does not return expected output. Wait for sometime, restart the kernel and try to run program again.

In [ ]:
pip install requests beautifulsoup4

In [1]:
import requests                       # Importing the necessary packages
from bs4 import BeautifulSoup         # Importing BeautifulSoup library
import re
import sqlite3



def data_scrap(ticker):               # Function definition for ticker
    
    PClose=''                         # Creating the string for the value set needed
    Open=''                           # Creating the string for the value set needed
    Volume=''                         # Creating the string for the value set needed
    scrapeddataset=[] 

    fin_url = 'https://finance.yahoo.com/quote/' + ticker + '?p=' + ticker   # Yahoo url with ticker symbol
    fin_req = requests.get(fin_url)                                          # Sending request for url

    fin_page = BeautifulSoup(fin_req.text, 'html.parser')                    # Scrap html page using beatifulsoup

    fin_data = fin_page.find_all(class_="Bxz(bb) Bdbw(1px) Bdbs(s) Bdc($seperatorColor) H(36px)")  # Filtering data of the html page using class
    
    for val1 in fin_data:  
        scrapeddataset += [a.text for a in val1.find_all("span")] 

    for val2 in range(len(scrapeddataset)):               # Accessing Open Price, Prev Close Price, Volume and storing them in variables

        if(scrapeddataset[val2] == "Previous Close"): 
            PClose = scrapeddataset[val2+1]

        if(scrapeddataset[val2] == "Open"): 
            Open = scrapeddataset[val2+1] 

        if(scrapeddataset[val2] == "Volume"): 
            Volume = scrapeddataset[val2+1]
            Volume = re.sub(r'[^\w\s]','',Volume) 
    
    return(PClose,Open,Volume)


# Main Program


try:
    conn=sqlite3.connect('StocksDatabase.sqlite')   # Connecting SQL Database
    curr=conn.cursor()
    curr.execute('DROP TABLE IF EXISTS StocksTable')
    curr.execute('CREATE TABLE StocksTable(Ticker TEXT, Open_Price REAL, Prev_Price REAL, Volume INTEGER)')  #Assigning variable 'type' to the variables
                                                   
    f=open("stocks.csv",'w+')                      # Creating file name and initializing in write mode
                                                   
    comp_req = requests.get('https://money.cnn.com/data/hotstocks/')  #Using the requests library get the url for scrapping
                                                   
    comp_page = BeautifulSoup(comp_req.text, 'html.parser')  #Using beautifulsoup library , scrapping the page using html.parser
                                                   
    comp_data = comp_page.find_all(class_="wsod_dataTable wsod_dataTableBigAlt")  #Using the find_all function filtering out all the data present with class specified 

    values =[]
    title = ['Most Actives'] 
    
    for data in comp_data: 
        
        #Extracting all the needed data and values from the scrapped webpage using find_all func through for loop
                                                   
        [tmp.extract() for tmp in data.find_all(class_ = "wsod_aRight")]  
        abbr = [a.text for a in data.find_all(class_= "wsod_symbol")] 
        name = [a.text for a in data.find_all("span")] 
        values += [dict(zip(abbr, name))]                        # Arranging the data set using dict func

       
    finaldata = dict(zip(title, values))         # This list contains company ticker symbols and name sorted using dict func 

    space=' '
    count=0
    
    conn=sqlite3.connect('StocksDatabase.sqlite')
    curr=conn.cursor()
    curr.execute('DROP TABLE IF EXISTS StocksTable')
    curr.execute('CREATE TABLE StocksTable(Ticker TEXT, Open_Price REAL, Prev_Price REAL, Volume INTEGER)')

    print("Which stock are you interested in:")

    while count<=2:
        
        fd=finaldata[title[count]]
        title_val=(title[count])
        print('\n',title_val,'\n') 
        
        f.write(title_val)      # Storing titles: Most Actives in file.
        f.write('\n\n')
        
        for i in fd:

            print(i,fd[i])   # Printing ticker symbol and Company name

            #print(i)
            PClose,Open,Volume = data_scrap(i)  # Call function to scrap data based on ticker symbol
                        
            outputs=i,',',fd[i],',',PClose,',',Open,',',Volume,'\n'
            add_space=space.join(outputs)

            f.write(add_space)
            
            curr.execute('INSERT INTO StocksTable (Ticker, Open_Price, Prev_Price, Volume) VALUES (?,?,?,?)', (fd[i], Open, PClose, Volume))     #Invoking Data into StocksTable table of the database

        count=count+1
    conn.commit()
    curr.close()
    f.close()          # Closing file here

    while True:
        temp_val = False

        symbol_inp = input("\nEnter your Interested Company :  ")    #User needs to input the company name 
        symbol_inp = symbol_inp.upper()
        
        file_open=open('stocks.csv')                                 # Open Stocks file
        words=file_open.read().strip().split('\n')
        for a in words:
            line=a.split(',')
            
            for b in line:
                symbol=b.strip()

                    #print(ask)
                if (symbol==symbol_inp):
                    PClose,Open,Volume = data_scrap(symbol_inp)             # Call scrap function again to get data from website
                    
                    print('\n')
                    print("The data for ",line[0].strip(),line[1].strip(),"is the following:\n")
                    print(line[0].strip(),line[1].strip())
                    print('Previous Close :',  PClose)                  # Print value of PreviousClose
                    print('Open Price : ', Open)                        # Print value of Open Price
                    print('Volume : ', Volume)                          # Print value of Volume
                    temp_val = True
                    break
                    
            if (symbol==symbol_inp):
                break

        if temp_val:
            break
        else:
            print ("\nYou did not enter correct ticker symbol from the list.")
except:
    print("Please run the code again.")

Which stock are you interested in:

 Most Actives 

GE General Electric Co
F Ford Motor Co
CCL Carnival Corp
BAC Bank of America Corp
DAL Delta Air Lines Inc
NCLH Norwegian Cruise Line Holdings Ltd
MRO Marathon Oil Corp
OXY Occidental Petroleum Corp
T AT&T Inc
BA Boeing Co
Please run the code again.
